In [ ]:
import codecs


In [ ]:
import os

os.chdir('../')

In [ ]:
genres = os.listdir()
genres = [genre for genre in genres if genre.find('.') == -1]
genres

In [ ]:
pwd

In [ ]:
genre_dict = {}

for genre in genres:
    os.chdir(genre)
    filenames = os.listdir()
    filenames = [filename.replace('_dialog.txt', '') for 
                 filename in filenames if filename.find('.txt') != -1]
    
    genre_dict[genre] = filenames
    os.chdir('../')

In [ ]:
genre_dict

In [ ]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.
    
    - Removes stage directions, some proper pronouns, and special characters 
    - Removes punctuation
    - Lowering text
    """
     
#     # remove the characters [\], ['] and ["]
#     text = text.replace('C', "", text)    
    text = re.sub(r"\'", "", text)    
#     text = re.sub(r"\"", "", text)    
    
    # remove numbers
    text = re.sub('\d', '', text)

    
#     text = (text.replace('\n', ' ')
#                 .replace('CUT TO:', '')
#                 .replace('PAUSE', '')
#                 .replace('TITLE:', ''))
    
    # remove special characters / words
    text = text.replace('\n', ' ')
    text = text.replace('CUT TO:', '')
    text = text.replace('PAUSE', '')
    text = text.replace('TITLE:', '')
    text = text.replace('(CONT\'D)', '')
    text = text.replace('(V.O.)', '')
    text = text.replace('  ', ' ') 
    text = text.replace('DISSOLVE TO', '')
    text = text.replace('THE END', '')
    text = text.replace('INTERCUT WITH', '')
    text = text.replace('CONTD', '')
    text = text.replace('contd', '')
    text = text.replace('CUT TO', '')
    text = text.replace('Postman', '')
    text = text.replace('shrek final', '')
    text = text.replace('screening', '')
    text = text.replace('screening script', '')
    text = text.replace('shrek', '')
    text = text.replace('final screening', '')
    text = text.replace('revision', '')
    text = text.replace('final revision', '')
    text = text.replace('www', '')
    text = text.replace('screentalk org', '')
    text = text.replace('org', '')
    text = text.replace('screentalk', '')
    text = text.replace('www screentalk', '')

    
    # replace punctuation characters with spaces
    filters = '!"\'#$%&()*+,-./;<=>?@[\\]^_`{|}~\t\n•'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    #replace headers
    text = text.replace('Written by', '')
    text = text.replace('Screenplay by', '')
    text = text.replace('Guardians Screening Script', '')
    
    #remove multiple whitespaces
    text = re.sub(' +', ' ', text)
    

    return text

In [ ]:
import re
import pandas as pd

genre_file_char_dict = {}

os.chdir('/Users/nicksherwin/Desktop/Project_github/Project 4/dialogs')

sorted_genre_dict_items = sorted(genre_dict.items(), key = lambda x:x[0])
good_char_list = 'all_characters.txt'

def process_df():
    for genre, titles in sorted_genre_dict_items:
        print(genre)
        genre_file_char_dict[genre] = {}
        titles = sorted(titles)
        for title in titles:
            print('\t' + title)
            genre_file_char_dict[genre][title] = {}
            os.chdir(genre)
            with codecs.open(title + '_dialog.txt', 'r', 'utf8') as fp:
                f = fp.read()
                text = clean_text(f)
                #TODO: describe regex for readee
                speaker = re.findall(r"([A-Z]+\s?[A-Z]+[^a-z0-9\W])",text)
                dialogue = re.compile('[A-Z]+\s?[A-Z]+[^a-z0-9\W]').split(text)[1:]
                movie_df = pd.DataFrame(list(zip(speaker, dialogue)), columns=['Character', 'Dialogue'])
                movie_df['Dialogue'] = movie_df['Dialogue'].str.lower()
                movie_df = movie_df.groupby('Character').filter(lambda x: len(x) > 100)
                movie_df = movie_df[movie_df.Dialogue.str.count(' ') > 3]
                char_list = movie_df.Character.unique()

            os.chdir('../')

            for idx, char in enumerate(char_list):
                if char in good_char_list:
                    char_df = movie_df[movie_df['Character'] == char] #creates separate df for each char
                    genre_file_char_dict[genre][title][char] = char_df

process_df()



In [ ]:
# import pprint as pp
# pp.pprint(genre_file_char_dict)

In [ ]:
len(genre_file_char_dict)

In [ ]:
genre_dict['Action']

### IBM Watson

In [15]:
from ibm_watson import PersonalityInsightsV3
import json
import os

In [16]:
url = 'https://gateway.watsonplatform.net/personality-insights/api'
apikey = 'yourapikey'
service = PersonalityInsightsV3(url=url, iam_apikey=apikey, version='2017-10-13' )

In [17]:
text = 'a wizard is never late frodo baggins nor is he early he arrives precisely when he means to. you didnt think id miss your uncle bilbos birthday   what everything far too eager and curious for a hobbit most unnatural   well what can i tell you life in the wide world goes on much as if it has past age full of its own comings and goings scarcely even aware of the existence of hobbits. ah the long expected party so how is the old rascal i hear its going to be a party of special magnificence   oh well that should please him   good gracious me   if youre referring to the incident with the dragon i was barely involved all i did was give your uncle a little nudge out the door '

In [18]:
profile = service.profile(text, accept='application/json').get_result()

In [19]:
print(json.dumps(profile, indent=2))

{
  "word_count": 131,
  "word_count_message": "There were 131 words in the input. We need a minimum of 600, preferably 1,200 or more, to compute statistically significant estimates",
  "processed_language": "en",
  "personality": [
    {
      "trait_id": "big5_openness",
      "name": "Openness",
      "category": "personality",
      "percentile": 0.792085729681743,
      "significant": true,
      "children": [
        {
          "trait_id": "facet_adventurousness",
          "name": "Adventurousness",
          "category": "personality",
          "percentile": 0.13645531288668483,
          "significant": true
        },
        {
          "trait_id": "facet_artistic_interests",
          "name": "Artistic interests",
          "category": "personality",
          "percentile": 0.9093237152104261,
          "significant": true
        },
        {
          "trait_id": "facet_emotionality",
          "name": "Emotionality",
          "category": "personality",
          "percen

In [192]:
import pickle

with open('characters.pkl', 'wb') as f:
    pickle.dump(genre_file_char_dict, f)

In [ ]:
from requests.exceptions import Timeout

In [ ]:
# with open(...) as f:
#     pickle.load(f)



genre_file_char_dict = pickle.load(open('genre_file_char_dict.pkl', 'rb'))

timeout_todo = []

for genre, titles in genre_file_char_dict.items():
    #print(genre)
    for title in titles:
        #print('\t' + title)
        chars = genre_file_char_dict[genre][title]
        for char, df in chars.items():
            #print('\t\t' + char)
            text = df['Dialogue'].values
            
            combined_text = ''
            for line in text:
                combined_text += line + '\n'
                
            if len(combined_text.split(' ')) >= 100:
                try:
                    profile = service.profile(combined_text, accept='application/json').get_result()
                except Timeout:
                    todo = dict(genre=genre, title=title, char=char, combined_text=combined_text, profile=profile)
                    timeout_todo.append(todo)
                genre_file_char_dict[genre][title][char] = {'text': combined_text, 
                                                            'profile': profile}

In [7]:
genre_file_char_dict['Sci-Fi']['pitchblack']['RIDDICK']['profile']

{'word_count': 1494,
 'processed_language': 'en',
 'personality': [{'trait_id': 'big5_openness',
   'name': 'Openness',
   'category': 'personality',
   'percentile': 0.8994757663299562,
   'significant': True,
   'children': [{'trait_id': 'facet_adventurousness',
     'name': 'Adventurousness',
     'category': 'personality',
     'percentile': 0.6420000977763174,
     'significant': True},
    {'trait_id': 'facet_artistic_interests',
     'name': 'Artistic interests',
     'category': 'personality',
     'percentile': 0.600776224263463,
     'significant': True},
    {'trait_id': 'facet_emotionality',
     'name': 'Emotionality',
     'category': 'personality',
     'percentile': 0.457676725012927,
     'significant': True},
    {'trait_id': 'facet_imagination',
     'name': 'Imagination',
     'category': 'personality',
     'percentile': 0.6863849428831956,
     'significant': True},
    {'trait_id': 'facet_intellect',
     'name': 'Intellect',
     'category': 'personality',
     

In [191]:
genre_file_char_dict.keys()

dict_keys(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [ ]:
combined = ''
for line in text:
    combined += line

In [4]:
# # Step 1
# import pickle
 
# # Step 2
# with open('dialogues.pickle', 'rb') as config_dictionary_file:
 
#     # Step 3
#     genre_file_char_dict = pickle.load(config_dictionary_file)
 
#     # After config_dictionary is read from file
#     # print(config_dictionary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Twitter + Comparison Test 

In [20]:
import twitter 
import tweepy
from ibm_watson import PersonalityInsightsV3
import json
import os


#This function is used to receive and analyze
#the last 200 tweets of a Twitter handle using
#the Watson PI API

def analyze(handle):

    #The Twitter API credentials
    consumer_key = 'ah8Gt84uDPpgVMN6KfBggWJJf'
    consumer_secret = 'lUVNB0YXd9HxVZKtCKu8EXERpzVgN2gFyXhoSa1gWVeYHM8TJD'
    access_token = '751451301876301824-RLBN7ivBGjWI4HURfj0Xll5sSdutAT4'
    access_token_secret = 'KLKAeUhlgEQ63dqYRO9ukAMdUnGybttKyt3HZiPVh6O3j'

    # Authorize our Twitter credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    #Retrieving the last 200 tweets from a user
    tweets = api.user_timeline(screen_name=handle, 
                               # 200 is the maximum allowed count
                               count=200,
                               include_rts = False,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               tweet_mode = 'extended'
                               )

    #Putting all 200 tweets into one large string called "text"
    text = "" 
    for s in tweets:
        if (s.lang =='en'):
                text += s.full_text
                
    #IBM Watson Credentials
    url = 'https://gateway.watsonplatform.net/personality-insights/api'
    apikey = 'Wcp-qDUY7OKpM6EUnA_15Aj4OzyAgGkxnq6VFfJR3joa'
    service = PersonalityInsightsV3(url=url, iam_apikey=apikey, version='2017-10-13' )
    
    #Analyzing the 200 tweets with the Watson PI API
    profile = service.profile(text, accept='application/json').get_result()
    
    #Returning the Watson PI API results
    return profile

In [ ]:
print(json.dumps(profile, indent=2))

In [39]:
user_personality = analyze(user_handle)['personality']

In [41]:
import pprint as pp
pp.pprint(user_personality)

[{'category': 'personality',
  'children': [{'category': 'personality',
                'name': 'Adventurousness',
                'percentile': 0.7999279190163954,
                'significant': True,
                'trait_id': 'facet_adventurousness'},
               {'category': 'personality',
                'name': 'Artistic interests',
                'percentile': 0.7672896664166076,
                'significant': True,
                'trait_id': 'facet_artistic_interests'},
               {'category': 'personality',
                'name': 'Emotionality',
                'percentile': 0.4922131506094467,
                'significant': True,
                'trait_id': 'facet_emotionality'},
               {'category': 'personality',
                'name': 'Imagination',
                'percentile': 0.05305863819079881,
                'significant': True,
                'trait_id': 'facet_imagination'},
               {'category': 'personality',
                'name': 'In

In [55]:
user_personality_vector = [0,0,0,0,0]
for idx, trait in enumerate(user_personality):
    user_personality_vector[idx] = trait['percentile']
    print(trait['name'], trait['percentile'])

Openness 0.8032122028835114
Conscientiousness 0.972882820949175
Extraversion 0.19172137712959303
Agreeableness 0.7044222559294065
Emotional range 0.6618106148466844


In [48]:
personality_vector

[0.8032122028835114,
 0.972882820949175,
 0.19172137712959303,
 0.7044222559294065,
 0.6618106148466844]

In [43]:
celebrity_personality = analyze(celebrity_handle)['personality']

In [56]:
celebrity_personality_vector = [0,0,0,0,0]
for idx, trait in enumerate(celebrity_personality):
    celebrity_personality_vector[idx] = trait['percentile']
    print(trait['name'], trait['percentile'])

Openness 0.9620227509995902
Conscientiousness 0.951486686172113
Extraversion 0.9094196983400185
Agreeableness 0.4876665693437961
Emotional range 0.12252583871055339


In [52]:
personality_vector

[0.9620227509995902,
 0.951486686172113,
 0.9094196983400185,
 0.4876665693437961,
 0.12252583871055339]

In [183]:
def most_similar_character(twitter_handle, user_defined_genre):
    user_personality = analyze(twitter_handle)['personality']

    user_personality_vector = [0,0,0,0,0]

    for idx, trait in enumerate(user_personality):
        user_personality_vector[idx] = trait['percentile']
    #     print(trait['name'], trait['percentile'])


    best_char_personality_vector = [0,0,0,0,0]
    best_char_title = 'No Title Match'
    best_char_name = 'No Character Match'
    best_similarity_score = 0

    titles = genre_file_char_dict[user_defined_genre].keys()

    for title in titles:
        chars = genre_file_char_dict[user_defined_genre][title].keys()
        for char in chars:
            try:
                profile = genre_file_char_dict[user_defined_genre][title][char]['profile']
                personality = profile['personality']
                char_personality_vector = [0,0,0,0,0]

                for idx, trait in enumerate(personality):
                    char_personality_vector[idx] = trait['percentile']

                similarity_score = round(cosine_similarity((user_personality_vector, 
                                                            char_personality_vector))[0][1], 3)

                if similarity_score > best_similarity_score:
                    best_char_personality_vector = char_personality_vector
                    best_char_title = title
                    best_char_name = char
                    best_similarity_score = similarity_score
            except:
                continue
#                 print(genre, title, char)
#                 print('\n')

    print('Genre: ' + genre + '\n' +
        'Title: ' + best_char_title + '\n'
         + 'Character Name: ' + best_char_name + '\n'
         + 'Similarity Score: ' + str(best_similarity_score) + '%')

In [190]:
genres = genre_file_char_dict.keys()


#cheers to Nick Sherwin's project working on his birthday

for genre in genres:
    most_similar_character('@realDonaldTrump', genre)
    print('\n')

Genre: Action
Title: italianjobthe
Character Name: CHARLIE
Similarity Score: 0.995%


Genre: Adventure
Title: italianjobthe
Character Name: CHARLIE
Similarity Score: 0.995%


Genre: Animation
Title: cars2
Character Name: FINN
Similarity Score: 0.96%


Genre: Comedy
Title: majorleague
Character Name: BROWN
Similarity Score: 0.986%


Genre: Crime
Title: italianjobthe
Character Name: CHARLIE
Similarity Score: 0.995%


Genre: Drama
Title: afewgoodmen
Character Name: ROSS
Similarity Score: 0.992%


Genre: Family
Title: riseoftheguardians
Character Name: NORTH
Similarity Score: 0.93%


Genre: Fantasy
Title: meetjoeblack
Character Name: DREW
Similarity Score: 0.965%


Genre: Horror
Title: jaws
Character Name: BRODY
Similarity Score: 0.984%


Genre: Musical
Title: sisteract
Character Name: MOTHER SUPERIOR
Similarity Score: 0.956%


Genre: Mystery
Title: afewgoodmen
Character Name: ROSS
Similarity Score: 0.992%


Genre: Romance
Title: outofsight
Character Name: MAURICE
Similarity Score: 0.986%


In [177]:
# deleting Film Noir key
if 'Film-Noir' in genre_file_char_dict: del genre_file_char_dict['Film-Noir']


In [180]:
genre_file_char_dict.keys()

dict_keys(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
round(cosine_similarity((user_personality_vector, celebrity_personality_vector))[0][1], 3)

0.841

In [194]:
most_similar_character('@screenshotart', 'Family')

Genre: Western
Title: kungfupanda
Character Name: SHIFU
Similarity Score: 0.991%
